In [1]:
import prior

dataset = prior.load_dataset("procthor-10k")
dataset

Fetching reference HEAD


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 23407.67it/s]


DatasetDict(
    train=Dataset(
    dataset=procthor-dataset,
    size=10000,
    split=train
),
    val=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=val
),
    test=Dataset(
    dataset=procthor-dataset,
    size=1000,
    split=test
)
)

In [2]:
from ai2thor.controller import Controller


house = dataset["train"][3]
controller = Controller(scene=house, snapToGrid=False, rotateStepDegrees=30, renderInstanceSegmentation=True)
event = controller.step("Pass")
spawn = event.metadata["agent"]["position"]

In [3]:
from rl import PPO, CLIPNovelty, ClipEnv, ActorCritic
from models import FrozenResNetEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetEncoder(project_to_out_dim=False)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = CLIPNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [4]:
from rl import inference, teleport
import torch

def get_distribution(ppo, obs_seq, actions_seq, actor_critic):
    dist = torch.distributions.Categorical(probs=torch.tensor([0.5, 0.25, 0.25]))
    return dist

all_obs = []
for i in range(12):
    event = teleport(controller)
    init_pos = event.metadata["agent"]["position"]
    obs = inference(get_distribution=get_distribution, controller=controller, ppo=ppo, init_position=init_pos, env=clip_env, actor_critic=clip_actor_critic, plot=False, n=32)
    all_obs.append(torch.stack(obs, dim=0))

all_obs_tensor = torch.cat(all_obs, dim=0)
embedding = encoder(all_obs_tensor.unsqueeze(0)).squeeze(0)

from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=FEAT_DIM)
pca.fit(embedding.detach().cpu().numpy())
W = pca.components_.astype(np.float32)

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

...

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

<Figure size 200x6400 with 0 Axes>

In [5]:
from rl import PPO, SegmentationNovelty, ClipEnv, ActorCritic, train
from models import FrozenResNetPCAEncoder, SlidingWindowTransformerActor, SlidingWindowTransformerCritic
from cons import FEAT_DIM, NUM_ACTIONS, DEVICE

ENTROPY_COEF = 0.05

ppo = PPO(ENTROPY_COEF)
encoder = FrozenResNetPCAEncoder(FEAT_DIM, torch.from_numpy(W), device=DEVICE)
actor = SlidingWindowTransformerActor(FEAT_DIM, NUM_ACTIONS) # Not important
critic = SlidingWindowTransformerCritic(FEAT_DIM) # Not important
clip_novelty = SegmentationNovelty()
clip_env = ClipEnv(clip_novelty)
clip_actor_critic = ActorCritic(encoder, actor, critic)

In [6]:
train(controller, "segment_novelty", ppo, clip_env, clip_actor_critic, 1)

wandb: Currently logged in as: viriyadhika-putra (viriyadhika1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


wandb: ERROR Problem finishing run


HandleAbandonedError: 